In [53]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from graph_utils import graph_gen, find_alpha_lower_bound
from algorithms import IPLUX

# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# from algorithms import B_DPP, Falsone, C_SP_SG, DPD_TV

num_node = 20
num_edge = 190

np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Graph with 20 nodes and 190 edges


C:\Users\xtq\AppData\Roaming\Python\Python311\site-packages\networkx\drawing\nx_pylab.py:315: UserWarning: 

The arrowsize keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  draw_networkx_edges(G, pos, arrows=arrows, **edge_kwds)


(20, 20)


In [54]:
from problem import Synthetic

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3
parameters['p'] = 1
parameters['m'] = 5

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x*: {prob.x_star}\n')

generating a Synthetic problem: N=20, d=3, p=1, m=5
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), cc: (20,)
                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 16 11:58:39 PM: Your problem has 60 variables, 26 constraints, and 0 parameters.
(CVXPY) Jan 16 11:58:39 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 11:58:39 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 11:58:39 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 11:58:39 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
---------------

In [55]:
MAX_ITER = 500

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
rho_choice = [0.1, 0.5, 2, 5]
alpha_choice = [alpha_lower]
rho_choice = [5]


np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
        for i in range(MAX_ITER):
            alg.step()

self.prob minimize param23807 @ var23806 + norm1(var23806) + 1.5 @ QuadForm(var23806 + -param23808, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + 0.1 @ quad_over_lin(param23809 @ var23806, 1.0) + param23810 @ var23806 + param23811[0] @ QuadForm(var23806, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + Promote(-2.0, (3,)) @ param23812 @ var23806
subject to QuadForm(var23806 + -param23814, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param23815[0]
reset
IPLUX alpha 3.0 rho 5, iter 0, obj err: 3.08e+00, cons vio: 0.00e+00
time 0.01, saved

IPLUX alpha 3.0 rho 5, iter 100, obj err: 6.17e-02, cons vio: 3.06e-01
time 6.87, saved

IPLUX alpha 3.0 rho 5, iter 200, obj err: 1.22e-02, cons vio: 1.35e-01
time 13.20, saved

IPLUX alpha 3.0 rho 5, iter 300, obj err: 9.80e-02, cons vio: 1.44e-01
time 19.48, saved

IPLUX alpha 3.0 rho 5, iter 400, obj err: 2.28e-02, cons vio: 1.12e-01
time 25.78, saved

IPLUX alpha 3.0 rho 5, iter 500, obj err: 6.72e-03, cons vio: 

In [56]:
# =========================================================================== |
# ---------------------------------- Plot ----------------------------------- |
# =========================================================================== |

from plot_utils import MyFigure


# alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
# rho_choice = [0.1, 0.5, 2, 5, 10]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
# marker = [".", "o", "^", "s", "p", "P", "*"]
linestyle = ['-', '--', '-.', ':', '']



# ================================ objective error============================
obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel=r'$|\sum_{i=1}^N f_i(x_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)|$',
                            yscale='log')
# obj_err_figure.add_line("IPLUX", alg.obj_err_log)
for rho in rho_choice[0:4]:
    for alpha in alpha_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'IPLUX_a{alpha}_r{rho}'
        filename = 'log/' + prefix +'_oe.txt'
        obj_err_figure.add_line_file(prefix, filename, style=c+ls)
obj_err_figure.paint(MAX_ITER=MAX_ITER)


# ================================= constraint violation ====================
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='constraint violation',
                            yscale='log')
for rho in rho_choice[0:4]:
    for alpha in alpha_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'IPLUX_a{alpha}_r{rho}'
        filename = 'log/' + prefix +'_cv.txt'
        cons_vio_figure.add_line_file(prefix, filename, style=c+ls)
cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


# ================================= x distance ==============================
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='x distance',
                            yscale='linear')
for rho in rho_choice[0:4]:
    for alpha in alpha_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'IPLUX_a{alpha}_r{rho}'
        filename = 'log/' + prefix +'_xd.txt'
        x_dis_figure.add_line_file(prefix, filename, style=c+ls)
x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


In [ ]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


In [7]:
print(alg.W)
print(alg.H)

[[ 6.50e-01 0.00e+00 0.00e+00 0.00e+00  1.00e-01 0.00e+00  8.33e-02
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  8.33e-02 0.00e+00
  0.00e+00 0.00e+00 0.00e+00  8.33e-02 0.00e+00 0.00e+00]
 [0.00e+00  8.19e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00  1.25e-01 0.00e+00 0.00e+00 0.00e+00
   5.56e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  7.45e-01 0.00e+00  1.00e-01 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00  7.14e-02 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  8.33e-02]
 [0.00e+00 0.00e+00 0.00e+00  8.19e-01 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
   5.56e-02 0.00e+00 0.00e+00 0.00e+00  1.25e-01 0.00e+00]
 [ 1.00e-01 0.00e+00  1.00e-01 0.00e+00  6.33e-01 0.00e+00  8.33e-02
  0.00e+00 0.00e+00  8.33e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+0